In [12]:
#!add-apt-repository -y ppa:jonathonf/ffmpeg-4
#!apt update
#!apt install -y ffmpeg

#!pip install huggingface_hub
#!pip install datasets>=2.6.1
#!pip install git+https://github.com/huggingface/transformers
#!pip install librosa
#!pip install evaluate>=0.30
#!pip install jiwer
#!pip install gradio
#!pip install accelerate>=0.20.1

In [91]:
from dataclasses import dataclass
from typing import Any,Dict,List,Union
import torch

In [65]:
from huggingface_hub import notebook_login
notebook_login()

In [50]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0","ru",split="train+validation",token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0","ru",split="test",token=True)
common_voice = common_voice.remove_columns(["accent","age","client_id","down_votes","gender","locale","path","segment","up_votes"])

In [52]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Russian", task="transcribe")

In [53]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Russian", task="transcribe")

In [54]:
from datasets import Audio
common_voice = common_voice.cast_column("audio",Audio(sampling_rate=16000))

def prepare_dataset(batch):
  audio = batch["audio"]
  batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
  batch["labels"] = tokenizer(batch["sentence"]).input_ids
  return batch

common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"],num_proc=2)

Map (num_proc=2):   0%|          | 0/32491 [00:00<?, ? examples/s]

TimeoutError: ignored

In [58]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Use Data Collator to perform Speech Seq2Seq with padding
    """
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [60]:
import evaluate
metric = evaluate.load("wer")

In [61]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

In [62]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [101]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./whisper-base-ru',
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,)

ImportError: ignored

In [98]:
from transformers import Seq2SeqTrainer

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

RuntimeError: ignored

In [103]:
print('Start train')
trainer.train()
print('End train')